[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PabloOrazi/cuarto-de-economia/blob/main/Regla%20de%20Taylor.ipynb)

In [90]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider, Layout

# Taylor Rule: i = r* + pi* + a(pi - pi*) + b(y - y*)
# r* = neutral real rate, pi = inflation, pi* = target inflation, y-y* = output gap
def taylor_rule(r_star=2.0, pi_star=2.0, pi=2.0, output_gap=0.0, a=0.5, b=0.5):
    inflation_gap = pi - pi_star
    contrib_r_star = r_star
    contrib_pi = pi
    contrib_infl_gap = a * inflation_gap
    contrib_output_gap = b * output_gap

    total_rate = contrib_r_star + pi_star + contrib_infl_gap + contrib_output_gap

    labels = ["$r^*$\n(Tasa Natural)",
              "$\pi*$ \n(Inflación\ndeseada)",
              "$a(\pi-\pi^*)$ \n(Desvío \nInflación)",
              "$b(y-y^*)$ \n(Desvío \nActividad)"]
    values = [contrib_r_star,
              pi_star,
              contrib_infl_gap,
              contrib_output_gap,
              ]
    real_rate = total_rate - pi

    colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", ]
    bottoms = [0,
               contrib_r_star,
               contrib_r_star+pi_star,
               contrib_r_star+pi_star+contrib_infl_gap]
    cum_negative = [0,
                    min(0,contrib_r_star),
                    min(0, contrib_r_star) + min(0, pi_star),
                    min(0, contrib_r_star) + min(0, pi_star) + min(0, contrib_infl_gap)
                    ]
    max_positive = sum([max(0, value) for value in values])
    min_bottom = min(cum_negative) + min(0, contrib_output_gap)

    # Plot bars side-by-side
    fig, ax = plt.subplots(figsize=(8,6))
    # ax.bar(labels, values, color=colors)

    # Plot each component as a stacked bar
    for i, (label, value, bottom) in enumerate(zip(labels, values, bottoms)):
        ax.bar(
            label,
            value,
            bottom=bottom,
            label="Total",
            color=colors[i],
        )

    # Plot each component as a stacked bar
    for i, (label, value, bottom, bottom_neg) in enumerate(zip(labels, values, bottoms, cum_negative)):
        ax.bar(
            "Total",
            value,
            bottom=bottom if value > 0 else bottom_neg,
            label=label,
            color=colors[i],
        )
    # Make a star at location "Total" and total_rate
    ax.scatter(
            "Total",
            total_rate,
            label="Total",
            color="black",
        )

    ax.bar(
            "Tasa Real",
            real_rate,
            bottom=0,
            label="Real Rate",
            color="black",
        )

    # Annotate values on top of bars
    for i, (v, b) in enumerate(zip(values, bottoms)):
        ax.text(i, b + max(v, 0) + 0.2, f"{v:.2f}", ha='center')

    # Annotate total rate in max_positive location for "Total"
    ax.text("Total", max_positive + 0.2, f"{total_rate:.2f}", ha='center')

    # Annotate total rate in max_positive location for "Total"
    ax.text("Tasa Real", real_rate + 0.2, f"{real_rate:.2f}", ha='center')

    # Make horizontal line at 0
    ax.axhline(y=0, color='black', linestyle='-')

    ax.set_ylim(-np.ceil(abs(min_bottom)), max(6, np.ceil(total_rate)))
    ax.set_ylabel("Tasa de interés (%)")
    ax.set_title("Regla de Taylor: i = r* + pi* + a(pi - pi*) + b(y - y*)")
    plt.show()

# Uniform style for aligned sliders
slider_style = {'description_width': '200px'}
slider_layout = Layout(width='600px')

# Interactive sliders
interact(taylor_rule,
         r_star=FloatSlider(value=2, min=0, max=4, step=0.1,
                            description="r* = (Tasa Natural)",
                            style=slider_style, layout=slider_layout),
         pi_star=FloatSlider(value=2, min=0, max=10, step=0.1,
                             description="pi* (Inflación deseada)",
                             style=slider_style, layout=slider_layout),
         pi=FloatSlider(value=2, min=0, max=4, step=0.1,
                        description="pi (Inflación Realizada)",
                        style=slider_style, layout=slider_layout),
         output_gap=FloatSlider(value=0, min=-10, max=10, step=0.1,
                                description="y-y* (Brecha de Producto)",
                                style=slider_style, layout=slider_layout),
         a=FloatSlider(value=1.5, min=0, max=2, step=0.1,
                       description="a = Reacción al desvío de Inflación",
                       style=slider_style, layout=slider_layout),
         b=FloatSlider(value=0.5, min=0, max=2, step=0.1,
                       description="b = Reacción al desvío de Actividad",
                       style=slider_style, layout=slider_layout))

interactive(children=(FloatSlider(value=2.0, description='r* = (Tasa Natural)', layout=Layout(width='600px'), …

<function __main__.taylor_rule(r_star=2.0, pi_star=2.0, pi=2.0, output_gap=0.0, a=0.5, b=0.5)>

In [91]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Baseline demand and supply functions
def demand(Q, a=20, b=1):
    return a - b*Q

def supply(Q, c=2, d=1):
    return c + d*Q

# Function to update the plot
def plot_market(shift_demand=0, shift_supply=0):
    Q = np.linspace(0, 20, 100)

    # Original curves
    P_demand = demand(Q)
    P_supply = supply(Q)

    # Shifted curves
    P_demand_shifted = demand(Q, a=20+shift_demand)
    P_supply_shifted = supply(Q, c=2-shift_supply)

    # Original equilibrium
    Q_eq = (20 - 2) / (1 + 1)
    P_eq = demand(Q_eq)

    # New equilibrium
    Q_new = (20 + shift_demand - (2 - shift_supply)) / (1 + 1)
    P_new = demand(Q_new, a=20+shift_demand)

    # Plot
    plt.figure(figsize=(8,6))
    plt.plot(Q, P_demand, label="Demanda Original", color="blue")
    plt.plot(Q, P_supply, label="Oferta Original", color="red")
    plt.plot(Q, P_demand_shifted, '--', label="Demanda desplazada", color="blue")
    plt.plot(Q, P_supply_shifted, '--', label="Oferta desplazada", color="red")

    # Mark equilibrium points
    plt.scatter(Q_eq, P_eq, color="black", zorder=5)
    plt.text(Q_eq+0.3, P_eq, f"E0 ({Q_eq:.1f}, {P_eq:.1f})")

    plt.scatter(Q_new, P_new, color="green", zorder=5)
    plt.text(Q_new+0.3, P_new, f"E1 ({Q_new:.1f}, {P_new:.1f})")

    # Arrows showing shifts
    if shift_demand != 0:
        plt.arrow(Q_eq, P_eq, 0, shift_demand*0.5, head_width=0.3, head_length=0.3, fc='blue', ec='blue')
    if shift_supply != 0:
        plt.arrow(Q_eq, P_eq, 0, -shift_supply*0.5, head_width=0.3, head_length=0.3, fc='red', ec='red')

    plt.xlabel("Cantidad")
    plt.ylabel("Precio")
    plt.title("Oferta y demanda con desplazamientos")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.show()

# Interactive sliders
interact(plot_market,
         shift_demand=FloatSlider(min=-10, max=10, step=1, value=0, description="Demand Shift"),
         shift_supply=FloatSlider(min=-10, max=10, step=1, value=0, description="Supply Shift"))


interactive(children=(FloatSlider(value=0.0, description='Demand Shift', max=10.0, min=-10.0, step=1.0), Float…

<function __main__.plot_market(shift_demand=0, shift_supply=0)>

In [98]:
from ipywidgets import HBox, interactive_output, VBox
# Create interactive outputs

plot_market_controls = {'shift_demand': FloatSlider(min=-10, max=10, step=1, value=0, description="Desplazamiento Demanda", style=slider_style, layout=slider_layout),
                        'shift_supply': FloatSlider(min=-10, max=10, step=1, value=0, description="Desplazamiento Oferta", style=slider_style, layout=slider_layout)}

plot_taylor_controls = {
         'r_star':FloatSlider(value=2, min=0, max=4, step=0.1,
                            description="r* = (Tasa Natural)",
                            style=slider_style, layout=slider_layout),
         'pi_star':FloatSlider(value=2, min=0, max=10, step=0.1,
                             description="pi* (Inflación deseada)",
                             style=slider_style, layout=slider_layout),
         'pi':FloatSlider(value=2, min=0, max=4, step=0.1,
                        description="pi (Inflación Realizada)",
                        style=slider_style, layout=slider_layout),
         'output_gap':FloatSlider(value=0, min=-10, max=10, step=0.1,
                                description="y-y* (Brecha de Producto)",
                                style=slider_style, layout=slider_layout),
         'a':FloatSlider(value=1.5, min=0, max=2, step=0.1,
                       description="a = Reacción al desvío de Inflación",
                       style=slider_style, layout=slider_layout),
         'b':FloatSlider(value=0.5, min=0, max=2, step=0.1,
                       description="b = Reacción al desvío de Actividad",
                       style=slider_style, layout=slider_layout)}

out1 = interactive_output(plot_market, plot_market_controls)
out2 = interactive_output(taylor_rule, plot_taylor_controls)

VBox([HBox([VBox(list(plot_market_controls.values()), layout=Layout(min_width="750px")),
            VBox(list(plot_taylor_controls.values()))]),
      HBox([out1, out2])])